In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Read the image
img = cv2.imread("crop1.jpg")

# Convert image BGR to RGB
original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Convert image to grayscale
gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

#1. Gaussian Blur
def Gaussian_blur(image):
    
    # Gaussian blur is applied to reduce noise in the image
    blurred_img = cv2.GaussianBlur(image, (7, 7), 1)

    return blurred_img

gaussian_blurred = Gaussian_blur(gray_img)

#2. Sobel Filter
def Sobel_filter(image):
    
    # Gradients in the x and y directions are calculated using Sobel filters
    gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, 3)
    gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, 3)

    sobel_img = cv2.addWeighted(np.abs(gx), 0.5, np.abs(gy), 0.5, 0)
    
    return sobel_img

sobel_filter = Sobel_filter(gaussian_blurred)


#3. Non-Maximum Suppression
def non_max_suppression(image, gradient_direction):
    height, width = image.shape
    suppressed_image = np.zeros((height, width), dtype=np.float32)
    
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            direction = gradient_direction[i, j]

            # Determine neighboring pixels
            if (0 <= direction < 22.5) or (157.5 <= direction <= 180):
                neighbor1 = image[i, j + 1]
                neighbor2 = image[i, j - 1]
            elif (22.5 <= direction < 67.5):
                neighbor1 = image[i + 1, j - 1]
                neighbor2 = image[i - 1, j + 1]
            elif (67.5 <= direction < 112.5):
                neighbor1 = image[i + 1, j]
                neighbor2 = image[i - 1, j]
            else:
                neighbor1 = image[i - 1, j - 1]
                neighbor2 = image[i + 1, j + 1]

            if (image[i, j] >= neighbor1) and (image[i, j] >= neighbor2):
                suppressed_image[i, j] = image[i, j]

    return suppressed_image


#4. Double Thresholding
def double_threshold(image, low_threshold, high_threshold):
    height, width = image.shape
    result = np.zeros((height, width), dtype=np.uint8)

    strong_pixel = 255
    weak_pixel = 50

    strong_i, strong_j = np.where(image >= high_threshold)
    weak_i, weak_j = np.where((low_threshold <= image) & (image < high_threshold))

    result[strong_i, strong_j] = strong_pixel
    result[weak_i, weak_j] = weak_pixel

    return result


#5. Hysteresis
def edge_tracking_by_hysteresis(image, weak_pixel, strong_pixel):
    height, width = image.shape

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            if image[i, j] == weak_pixel:
                if (image[i - 1:i + 2, j - 1:j + 2] == strong_pixel).any():
                    image[i, j] = strong_pixel
                else:
                    image[i, j] = 0

    return image


# Perform Canny Edge Detection
def canny_edge_detector(image, low_threshold, high_threshold):
    
    gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, 3)
    gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, 3)

    # Gradient magnitude
    gradient_magnitude = np.sqrt(gx ** 2 + gy ** 2)
    gradient_direction = np.arctan2(gy, gx) * 180 / np.pi

    # Ensure gradient direction is positive (0 to 180 degrees)
    gradient_direction[gradient_direction < 0] += 180

    # Non-Maximum Suppression
    suppressed_image = non_max_suppression(gradient_magnitude, gradient_direction)

    # Double Thresholding
    thresholded_image = double_threshold(suppressed_image, low_threshold, high_threshold)

    # Hysteresis
    edge_image = edge_tracking_by_hysteresis(thresholded_image, 50, 255)

    return edge_image

# Apply Canny Edge Detection
canny_edges = canny_edge_detector(sobel_filter, low_threshold=30, high_threshold=100)

# Create subplots to display images
fig, axes = plt.subplots(2, 4, figsize=(12, 8))

# Original Image
axes[0, 0].imshow(original_img, cmap='gray')
axes[0, 0].set_title('Original Image')
axes[0, 0].axis("off")

# Gaussian Blurred Image
gaussian_blurred = Gaussian_blur(gray_img)
axes[0, 1].imshow(gaussian_blurred, cmap='gray')
axes[0, 1].set_title('Gaussian Blurred')
axes[0, 1].axis("off")

# Sobel Filtered Image
sobel_filtered = Sobel_filter(gaussian_blurred)
axes[0, 2].imshow(sobel_filtered, cmap='gray')
axes[0, 2].set_title('Sobel Filter')
axes[0, 2].axis("off")

# Non-Maximum Suppression
suppressed = non_max_suppression(sobel_filtered, sobel_filtered)
axes[0, 3].imshow(suppressed, cmap='gray')
axes[0, 3].set_title('Non-Maximum Suppression')
axes[0, 3].axis("off")

# Double Thresholding
thresholded = double_threshold(suppressed, 30, 100)
axes[1, 0].imshow(thresholded, cmap='gray')
axes[1, 0].set_title('Double Thresholding')
axes[1, 0].axis("off")

# Hysteresis
hysteresis = edge_tracking_by_hysteresis(thresholded, 50, 255)
axes[1, 1].imshow(hysteresis, cmap='gray')
axes[1, 1].set_title('Hysteresis')
axes[1, 1].axis("off")

# Canny Edge Detection
axes[1, 2].imshow(canny_edges, cmap='gray')
axes[1, 2].set_title('Canny Edge Detection')
axes[1, 2].axis("off")

# Remove empty subplot
axes[1, 3].axis("off")

plt.tight_layout()
plt.show()